# Computation on Arrays: Broadcasting
--------------------

**Broadcasting** is a set of rules for applying binary ufuncs (e.g., addition, subtraction, multiplication, etc.) on arrays of **different sizes**.

### 1. Introducing Broadcasting
-------------

* For arrays of **the same size** binary operations are performed on an **element-by-element** basis:

In [ ]:
import numpy as np

In [ ]:
a = np.array([0, 1, 2])
b = np.array([5, 5, 5])
a + b

*  Broadcasting allows these types of binary operations to be performed on arrays of **different sizes**:

In [ ]:
a + 5

* Broadcasting for arrays of **higher dimension**. 

Observe the result when we add a one-dimensional array to a two-dimensional array:

In [ ]:
M = np.ones((3, 3))
M

In [ ]:
M + a

In [ ]:
a = np.arange(3)
b = np.arange(3)[:, np.newaxis]

print(a)
print(b)

In [ ]:
a + b

Just as before we stretched or broadcasted one value to match the shape of the other, here we've stretched *both* ``a`` and ``b`` to match a **common shape**, and the result is a two-dimensional array:

![Broadcasting Visual](broadcasting.png)

The light boxes represent the broadcasted values: again, this extra memory is **not actually** allocated in the course of the operation, but it can be useful conceptually to imagine that it is.

*  Broadcasting is the useful **mental model**:
   * We can think of this as an operation that stretches or duplicates the value ``5`` into the array ``[5, 5, 5]``, and adds the results
   * The advantage of NumPy's broadcasting is that this duplication of values does not actually take place, but it is  as we think about broadcasting.

### 2. Rules of Broadcasting
------------------

Broadcasting in NumPy follows a strict set of rules to determine the interaction between the two arrays:

- Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is *padded* with ones on its leading (**left**) side.
- Rule 2: If the shape of the two arrays does not match in any dimension, the array with shape **equal to 1** in that dimension is stretched to match the other shape.
- Rule 3: If in any dimension the sizes disagree and neither is equal to 1, an **error** is raised.

#### 2.1. Broadcasting example 1

In [ ]:
M = np.ones((2, 3))
a = np.arange(3)

The shape of the arrays are

- ``M.shape = (2, 3)``
- ``a.shape = (3,)``

We see by rule 1 that the array ``a`` has fewer dimensions, so we pad it on the left with ones:

- ``M.shape -> (2, 3)``
- ``a.shape -> (1, 3)``

By rule 2, we now see that the first dimension disagrees, so we stretch this dimension to match:

- ``M.shape -> (2, 3)``
- ``a.shape -> (2, 3)``

The shapes match, and we see that the final shape will be ``(2, 3)``:

In [ ]:
M + a

#### 2.2. Broadcasting example 2
Both arrays need to be broadcast:

In [ ]:
a = np.arange(3).reshape((3, 1))
b = np.arange(3)

Again, we'll start by writing out the shape of the arrays:

- ``a.shape = (3, 1)``
- ``b.shape = (3,)``

Rule 1 says we must pad the shape of ``b`` with ones:

- ``a.shape -> (3, 1)``
- ``b.shape -> (1, 3)``

And rule 2 tells us that we upgrade each of these ones to match the corresponding size of the other array:

- ``a.shape -> (3, 3)``
- ``b.shape -> (3, 3)``

Because the result matches, these shapes are compatible. We can see this here:

In [ ]:
a + b

#### 2.3. Broadcasting example 3

Two arrays are not compatible:

In [ ]:
M = np.ones((3, 2))
a = np.arange(3)

This is just a slightly different situation than in the first example: the matrix ``M`` is transposed.
How does this affect the calculation? The shape of the arrays are

- ``M.shape = (3, 2)``
- ``a.shape = (3,)``

Again, rule 1 tells us that we must pad the shape of ``a`` with ones:

- ``M.shape -> (3, 2)``
- ``a.shape -> (1, 3)``

By rule 2, the first dimension of ``a`` is stretched to match that of ``M``:

- ``M.shape -> (3, 2)``
- ``a.shape -> (3, 3)``

Now we hit rule 3–the final shapes do not match, so these two arrays are incompatible, as we can observe by attempting this operation:

In [ ]:
M + a

#### 2.4. Using  ``np.newaxis`` for **right**-side padding

In [ ]:
a[:, np.newaxis].shape

In [ ]:
M + a[:, np.newaxis]

Also note that while we've been focusing on the ``+`` operator here, these broadcasting rules apply to *any* binary ``ufunc``.
For example, here is the ``logaddexp(a, b)`` function, which computes ``log(exp(a) + exp(b))`` with more precision than the naive approach:

In [ ]:
np.logaddexp(M, a[:, np.newaxis])

### 3. Broadcasting in Practice
----------------------

ufuncs allow a NumPy user to remove the need to explicitly write slow Python loops. Broadcasting extends this ability.

#### 3.1. Centering an array

One commonly seen example is when centering an array of data.
Imagine you have an array of 10 observations, each of which consists of 3 values:

In [ ]:
X = np.random.random((10, 3))

In [ ]:
X

We can compute the mean of each feature using the ``mean`` aggregate across the first dimension:

In [ ]:
Xmean = X.mean(0)
Xmean

And now we can center ``X`` array (this is a broadcasting operation):

In [ ]:
X_centered = X - Xmean

To double-check that we've done this correctly, we can check that the centered array has near zero mean:

In [ ]:
X_centered.mean(0)

To within machine precision, the mean is now zero.

#### 3.2. Plotting a two-dimensional function

Broadcasting can be used to compute the function $z = f(x, y)$ across the grid:

In [ ]:
# x and y have 50 steps from 0 to 5
x = np.linspace(0, 5, 100)
y = np.linspace(0, 5, 100)[:, np.newaxis]

z = np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(z, origin='lower', extent=[0, 5, 0, 5],
           cmap='viridis')
plt.colorbar();

The result is a compelling visualization of the two-dimensional function.